In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import random

In [ ]:
sns.set(color_codes=True)
random.seed(1000)

## 1. The training dataset generation

Synthesized data, which is generated as 2-D  smaples according to two different Gaussian distributions, will be used.

Therefore, we need first define the mean and covariance of each Gaussian distribution.

In [ ]:
nums_train = 200 # set the number of the training data

In [ ]:
train_m1 = np.array([1, -1]) ## the mean of the 1-st Gaussian distribution
train_m2 = np.array([-1, 1])  ## the mean of the 2-nd Gaussian distribution
train_means = [train_m1, train_m2]

print("The means of the two Gaussian distributions:", train_means[0], train_means[1])

## define the two covariance matrices of the two Gaussian distributions
train_c1 = np.array([[1,0], [0,1]])
train_c2 = np.array([[1, 0], [0, 1]])
train_covs = [train_c1, train_c2] ## Each covariance matrix decides the shape of the Gaussian distribution

print("The covariance matrices of the two Gaussian distributions:\n", train_covs)

In [ ]:
def sample_generation(means, covs, nums):
    
    ''''
    input:
        - means: a list with 2 elements, and each represents the mean of a Gaussian distribution
        - covs: a list with 2 elements, and each represents the covariance matrix of a Gaussian distribution
        - nums: a scalar, the number of data points to be generated
    
    output: an array with the shape of (nums, 3)
    '''
    
    # sampling (nums//2) points from the 1st Gaussian distribution with means[0] and covs[0]
    x_1 = np.random.multivariate_normal(means[0], covs[0], nums//2)
    y_1 = np.zeros((nums//2, 1))
    
    # sampling (nums//2) points from the 2nd Gaussian distribution with mean[1] and covs[1] 
    x_2 = np.random.multivariate_normal(means[1], covs[1], nums//2)
    y_2 = np.ones((nums//2, 1))
    
    # In this demo, we define the shape of our data as (nums, 3)
    # The first two columns represent features, the 3rd column is the label.
    
    d1 = np.hstack((x_1, y_1))
    d2 = np.hstack((x_2, y_2))
    data = np.vstack((d1, d2))
    
    # Shuffle the data, or rearrange the order of the data
    index = [i for i in range(0, nums)]
    random.shuffle(index)
    data = data[index,:]
    return data

In [ ]:
train_data = sample_generation(train_means, train_covs, nums_train)

#  data visualization

In [ ]:
def data_visual(data):
    
    label_set = set(data[:,2].flatten())
    signs = ['r*', 'bo']
    
    for i,label in enumerate(zip(label_set)):
        index = np.where(data[:,2] == label)
        x = data[index, 0]
        y = data[index, 1]
        plt.plot(x, y, signs[i])
    
    return None

In [ ]:
data_visual(train_data)

## Logistic regression

### Sigmoid function

$$ h_\theta(x) = \frac{1}{1+ e^{-\theta^T{x}} } $$

In [ ]:
def sigmoid(x):
    
    '''
    input:
        - X: an array with the shape of (nums, 1)
    
    output: an array with the shape of (nums, 1)
    '''
    y = 1/(1 + np.exp(-x))
    
    return y

In [ ]:
def h(X, theta):
    
    '''
    input:
        - X: the training data array  with the shape of (nums, 2)
        - theta: the weight array with the shape of (2, 1)
    
    output: an array with the shape of (nums, 1)
    '''
    return sigmoid(np.dot(X, theta))

### Logistic Regression Cost Function

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m} [-y^{(i)}\log (h_\theta(x^{(i)})) - (1-y^{(i)})\log (1-h_\theta(x^{(i)})]
$$

### Gradient descent update

$$ \theta_j :=  \theta_j  - \alpha \sum_{i=1}^{m} h_\theta(x^{(i)} - y^{(i)}) x_j^{(i)} $$
    (simultaneously update all $\theta_j$)

In [ ]:
def object_fun(X, theta, y):
    '''
    input:
        - X: an array with the shape of (nums, 2), nums means the number of sample points, and 2 represents
            the dimension of the features.
        - theta: (2, 1) array.
        - y: an array with the shape of (nums, 1), containing labels of the sample points.
    
    output: a scalar
    '''
    
    nums_sample = X.shape[0]
    h_theta = h(X, theta)
    L = (1.0 / nums_sample) * ((-y).T.dot(np.log(h_theta)) - (1.0 - y.T).dot(np.log(1.0-h_theta)))
    grad = (1.0/nums_sample) * X.T.dot(h_theta - y)
    return L, grad

In [ ]:
def predict(X, theta):
    '''
    input:
        - X: an array with the shape of (nums, 2), nums means the number of sample points, and 2 represents
            the dimension of the features.
        - theta: (2, 1) array.
    
    output: a binary arrary with the shape of (nums, 1)
    '''
    
    # Perform sigmoid function first
    pred = h(X, theta)
    
    # The ouput of the sigmoid function is continuous, a threshold should be used for the classification task.
    # In this example, the threshold is set to 0.5.
    
    pred[pred >= 0.5] = 1 # Output values above 0.5 are set to 1。 
    pred[pred < 0.5] = 0 # Output values above 0.5 are set to 0.
    return pred

In [ ]:
def comp_accuracy(X, theta, y):
    '''
    input:
        - X: an array with the shape of (nums, 2), nums means the number of sample points, and 2 represents
            the dimension of features.
        - theta: (2, 1) array.
        - y: an array with the shape of (nums, 1), containing labels of sample points.
    
    output: a scalar
    '''
    
    pred = predict(X, theta)
    
    return np.mean(pred==y)

In [ ]:
## training data preparation
train_x, train_y = train_data[:, 0:2], train_data[:,2]
train_y = train_y.reshape(nums_train, 1)

## paramter settings
iterations = 1000  ## set the number of iterations for gradient descent
theta_init = np.random.rand(2,1)  ## set the initial value of the theta.

theta_old = theta_init
alpha = 0.005 ## the learning rate
loss_list = [] ## a list to record the loss at each iteration
acc_list = [] ## a list to record the accuracy at each iteration

for ite in range(0, iterations):
    
    ## compute the loss
    loss, grad = object_fun(train_x, theta_old, train_y)
    loss_list.append(loss[0,0])
    
    ## update theta
    theta_new = theta_old - alpha * grad
    
    ## compute the current accuracy
    acc_temp = comp_accuracy(train_x, theta_old, train_y)
    acc_list.append(acc_temp)
    
    theta_old = theta_new
    if (ite+1) % 100 == 0:
        print("The iteration {}, the loss is {:.4f}, the accuracy is {:.4f}".format((ite+1), loss[0,0], acc_temp))

theta_hat = theta_old  ## final updated paramters.

 ## Visualization for loss and accuracy

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
plt.plot(loss_list)
plt.xlabel("iteration")
plt.ylabel("loss")

plt.subplot(1,2,2)
plt.plot(acc_list)
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.ylim((0,1))

## Visualization for the decision boundary

In [ ]:
ss = 0.02  ## step size

# create a mesh to plot in
x_min, x_max = train_x[:, 0].min() - 1, train_x[:, 0].max() + 1
y_min, y_max = train_x[:, 1].min() - 1, train_x[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, ss),
                     np.arange(y_min, y_max, ss))

In [ ]:
# Plot the decision boundary. For visualization, we assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
temp = np.c_[xx.ravel(), yy.ravel()]
pred = predict(temp, theta_hat)
Z = pred.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.RdBu, alpha=0.6)
label_set = set(train_y.flatten())

# set the color we would like to assign to each point.
color_map = {1: (0, 0, .9), 0: (0.9, 0, 0)}
color = [color_map[i] for i in list(train_y.ravel())]
color = np.asarray(color)

# Plot the training points
for i,label in enumerate(zip(label_set)):
    index = np.where(train_y == label)
    data_x = train_x[index[0], 0]
    data_y = train_x[index[0], 1]
    c = color[index[0],:]
    plt.scatter(data_x, data_y,  c=c, edgecolors='black')
plt.axis('off')

## Test data generation

In [ ]:
nums_test = 100  # set the number of test samples 

test_m1 = np.array([0.5, -0.5]) ## mean of the 1-st Gaussian distribution
test_m2 = np.array([-0.5, 0.5])  ## mean of the 2-nd Gaussian distribuion
test_means = [test_m1, test_m2]

print("The means of two Gaussian distribution:", test_means[0], test_means[1])

## Define the  covariance matrice of the two Gaussian distributions
test_c1 = np.array([[0.5,0], [0,0.5]])
test_c2 = np.array([[0.5, 0], [0, 0.5]])
test_covs = [test_c1, test_c2] ## Each covariance matrix decides the shape of the Gaussian distribution
print("The covariance matrices of the two Gaussian distributions:\n", test_covs)

In [ ]:
test_data = sample_generation(test_means, test_covs, nums_test)

In [ ]:
# Split testing data and testing label
test_x, test_y = test_data[:, 0:2], test_data[:,2]
test_y = test_y.reshape(nums_test, 1)

In [ ]:
acc = comp_accuracy(test_x, theta_hat, test_y)

print("The testing accuracy is {:.4f}".format(acc))